# モデルの精度を確認するためのインターフェース

ver.2023/3/15

学習したモデルでTTSと非リアルタイムのVCを行い、モデルの精度を検証します。

​

In [ ]:
#@title ## 0 ノートブックの準備

#@markdown このノートブックのセットアップを行います。セルを実行してください。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#@markdown セルの実行が完了したら、次に進んでください。

#@markdown 「警告: このノートブックは Google が作成したものではありません。」といったポップアップが表示された場合、内容を確認して「このまま実行」を選択してください。このノートブックでは、外部へのデータ送信は一切行われません。

#debug用ディレクトリの作成
!rm -rf /mmvc-debug
!mkdir /mmvc-debug

#現在時刻の取得
import datetime
jst_delta = datetime.timedelta(hours=9)
JST = datetime.timezone(jst_delta, 'JST')
now = datetime.datetime.now(JST)
nowt = now.strftime('%Y%m%d%H%M%S')

#出力記録用カスタムマジック %%ccapture
from IPython import get_ipython
from IPython.core import magic_arguments
from IPython.core.magic import register_cell_magic
from IPython.utils.capture import capture_output

@magic_arguments.magic_arguments()
@magic_arguments.argument('output', type=str, default='', nargs='?')

@register_cell_magic
def ccapture(line, cell):
    args = magic_arguments.parse_argstring(ccapture, line)
    with capture_output() as outputs:
        get_ipython().run_cell(cell)
    if args.output:
        get_ipython().user_ns[args.output] = outputs
    
    outputs()

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
#git-lfs 事前学習済みモデルのダウンロードに使用
!apt install git-lfs
#os ディレクトリの存在確認に使用
import os
#sys Pythonの管理全般(クラッシュ、tracebackなど)
import sys
print("log: パッケージのインストールが完了しました。\n")


​

In [ ]:
%%ccapture one_prepare_notebook
#@title ## 1 Notebookの準備
#@markdown このNotebookの実行に必要なパッケージを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

#Pythonパッケージのインストール
print("log: パッケージのインストールを開始します。")
##ipywidgets UIの実装に使用
!pip install ipywidgets
import ipywidgets as widgets
#IPython.display 出力の消去に使用
from IPython.display import clear_output
#time waitコマンドの実装に使用
import time
print("log: パッケージのインストールが完了しました。\n")

​

In [ ]:
%%ccapture two_prepare_repo
#@title ## 2 リポジトリの準備
#@markdown リポジトリの準備を行います。

#@markdown まず、セルを実行してください。UIが表示されます。

#@markdown Colabを使用している場合
#@markdown * PlatformでColabを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、Google Driveのマイドライブ以下のパスを指定してください。よく分からない場合は、変更しなくとも構いません。変更しない場合は、マイドライブ直下に保存されます。
#@markdown * 「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。

#@markdown ローカルの場合
#@markdown * PlatformでLocalを選択してください。
#@markdown * PathでMMVC_Trainerの保存先となる、ローカルのパスを指定してください。

#@markdown 設定が完了したら、次へを押してください。


#---関数---
def mount_googledrive():
  print("log: Google Driveのマウントを開始します。")
  print("「このノートブックに Google ドライブのファイルへのアクセスを許可しますか？」といったポップアップが表示されますので、「Google ドライブに接続」を押下し、google アカウントを選択して、「許可」を選択してください。")
  time.sleep(2)
  print("info: 少し時間がかかります。このままお待ちください。")
  from google.colab import drive
  drive.mount('/content/drive')
  print("log: Google Driveのマウントが完了しました。\n")

#---関数終わり---

def first_view():
  #widgetsの構成
  global platform_input #グローバル指定
  global path_input #グローバル指定
  platform_input = widgets.Dropdown(options=["Colab", "Local"], value="Colab", description='Platform:', disabled=False)
  path_input = widgets.Text(value='/content/drive/MyDrive/MMVC_Trainer-v1.5.0.0_SiFiGAN', placeholder='/content/drive/MyDrive/MMVC_Trainer-v1.5.0.0_SiFiGAN', description='Path:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(platform_input, path_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global platform #グローバル指定
  platform = platform_input.value
  global path #グローバル指定
  path = path_input.value
  #Colabとそれ以外で処理を分ける
  if platform == "Colab":
    mount_googledrive() #Google Driveのマウント
    %cd $path
  elif platform == "Local":
    %cd $path

#実行
first_view()

​

In [ ]:
%%ccapture three_install_library
#@title ## 3 ライブラリのインストール
#@markdown 実行に必要なライブラリを導入します。

#@markdown 時間がかかりますので、気長にお待ちください。

!apt-get install espeak
!pip install -r requirements.txt

%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import create_dataset
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader
import torchaudio
import numpy as np 

import commons
import utils
from data_utils import TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from scipy.io.wavfile import write

​

In [ ]:
%%ccapture four_load_model
#@title ## 4 学習したモデルの読み込み
#@markdown 学習したモデルを読み込みます。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown CONFIG_PATH：configファイルのパス
#@markdown 作成したconfigファイル(json)を指定してください。  
#@markdown `configs/****.json` のような値になります。

#@markdown NET_PATH：学習したモデルのパス
#@markdown 作成されたモデル(pth)を指定してください。
#@markdown `logs/xxxxxxxx_xxxxx/G_xxxxx.pth` のような値になります。

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。

def first_view():
  #widgetsの構成
  global CONFIG_PATH_input #グローバル指定
  global NET_PATH_input #グローバル指定
  CONFIG_PATH_input = widgets.Text(value='configs/train_config.json', placeholder='configs/train_config.json', description='config_path:', disabled=False)
  NET_PATH_input = widgets.Text(value='logs/20220306_24000/G_best.pth', placeholder='logs/20220306_24000/G_best.pth', description='net_path:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(CONFIG_PATH_input, NET_PATH_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global CONFIG_PATH #グローバル指定
  CONFIG_PATH = CONFIG_PATH_input.value
  global NET_PATH #グローバル指定
  NET_PATH = NET_PATH_input.value
  #モデルの読み込み
  global hps
  hps = utils.get_hparams_from_file(CONFIG_PATH)
  global net_g
  net_g = SynthesizerTrn(
      spec_channels = hps.data.filter_length // 2 + 1,
      segment_size = hps.train.segment_size // hps.data.hop_length,
      inter_channels = hps.model.inter_channels,
      hidden_channels = hps.model.hidden_channels,
      upsample_rates = hps.model.upsample_rates,
      upsample_initial_channel = hps.model.upsample_initial_channel,
      upsample_kernel_sizes = hps.model.upsample_kernel_sizes,
      n_flow = hps.model.n_flow,
      dec_out_channels=1,
      dec_kernel_size=7,
      n_speakers = hps.data.n_speakers,
      gin_channels = hps.model.gin_channels,
      requires_grad_pe = hps.requires_grad.pe,
      requires_grad_flow = hps.requires_grad.flow,
      requires_grad_text_enc = hps.requires_grad.text_enc,
      requires_grad_dec = hps.requires_grad.dec
      )
  _ = net_g.eval()
  _ = utils.load_checkpoint(NET_PATH, net_g, True, None)
  print("log: モデルの読み込みが完了しました")

#実行
first_view()

​

In [ ]:
%%ccapture five_vc
#@title ## 5 学習したモデルで非リアルタイムVCを行う(学習に利用した音声ファイル)
#@markdown 非リアルタイムのVCを行います。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown SOURCE_SPEAKER_ID
#@markdown ソース話者のID

#@markdown SOURCE_WAVFILE
#@markdown ソース話者の音声ファイルのパス
#@markdown 学習に使用した音声ファイルのうちいずれか一つを指定してください。

#@markdown TARGET_ID
#@markdown ターゲット話者のID

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。
#@markdown 変換が行われ、変換後の音声が出力されます。

#@markdown VCの性能が悪い場合、学習不足か他に問題があります。


def first_view():
  #widgetsの構成
  global SOURCE_SPEAKER_ID_input #グローバル指定
  global SOURCE_WAVFILE_input #グローバル指定
  global TARGET_ID_input #グローバル指定
  global f0_scale_input #グローバル指定
  SOURCE_SPEAKER_ID_input = widgets.IntText(value='0', placeholder='0', description='SOURCE_SPEAKER_ID:', disabled=False)
  SOURCE_WAVFILE_input = widgets.Text(value='dataset/00_myvoice/wav/VOICEACTRESS100_001.wav', placeholder='dataset/00_myvoice/wav/VOICEACTRESS100_001.wav', description='SOURCE_WAVFILE:', disabled=False)
  TARGET_ID_input = widgets.IntText(value='101', placeholder='101', description='TARGET_ID:', disabled=False)
  f0_scale_input = widgets.FloatSlider(value='2.0', min='0.1', max='5.0', step='0.1', description='f0_scale:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(SOURCE_SPEAKER_ID_input, SOURCE_WAVFILE_input, TARGET_ID_input, f0_scale_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global SOURCE_SPEAKER_ID #グローバル指定
  SOURCE_SPEAKER_ID = SOURCE_SPEAKER_ID_input.value
  global SOURCE_WAVFILE #グローバル指定
  SOURCE_WAVFILE = SOURCE_WAVFILE_input.value
  global SOURCE_WAVFILE_id #グローバル指定
  SOURCE_WAVFILE_id = SOURCE_WAVFILE.split("/")[1]
  global SOURCE_WAVFILE_filename #グローバル指定
  SOURCE_WAVFILE_filename = SOURCE_WAVFILE.split("/")[-1].split(".")[0]
  global TARGET_ID #グローバル指定
  TARGET_ID = TARGET_ID_input.value
  global DUMMY #グローバル指定
  DUMMY = "dataset_etc/units/" + SOURCE_WAVFILE_id + "/" + SOURCE_WAVFILE_filename + ".npy"
  global F0 #グローバル指定
  F0 = "dataset_etc/F0/" + SOURCE_WAVFILE_id + "/" + SOURCE_WAVFILE_filename + ".npy"
  global CF0 #グローバル指定
  CF0 = "dataset_etc/cF0/" + SOURCE_WAVFILE_id + "/" + SOURCE_WAVFILE_filename + ".npy"
  global f0_scale #グローバル指定
  f0_scale = f0_scale_input.value

  with torch.no_grad():
    dataset = TextAudioSpeakerLoader(hps.data.training_files_notext, hps.data)
    data = dataset.get_audio_text_speaker_pair([SOURCE_WAVFILE, SOURCE_SPEAKER_ID, DUMMY, F0, CF0])
    data = TextAudioSpeakerCollate(
      sample_rate = hps.data.sampling_rate,
      segment_size = hps.train.segment_size,
      hop_size = hps.data.hop_length,
      df_f0_type = hps.data.df_f0_type,
      dense_factors = hps.data.dense_factors,
      upsample_scales = hps.model.upsample_rates,
      sine_amp = hps.data.sine_amp,
      noise_amp = hps.data.noise_amp,
      sine_f0_type = hps.data.sine_f0_type,
      signal_types = hps.data.signal_types,
      train = False,
      f0_factor = f0_scale,
  )([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src, f0, f0_lengths, sin, d, slice_id = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, f0, sid_src=sid_src, sid_tgt=sid_tgt1)[0][0,0].data.cpu().float().numpy()
    print("Original SID: %d" % sid_src.item())
    ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate,normalize = False))
    print("Converted SID: %d" % sid_tgt1.item())
    ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate,normalize = False))

#実行
first_view()

In [ ]:
%%ccapture six_vc
#@title ## 6 学習したモデルで非リアルタイムVCを行う(学習に利用していない音声ファイル)
#@markdown 非リアルタイムのVCを行います。

#@markdown まず、セルを実行してください。

#@markdown UIが表示されますので、以下の説明に従って設定を行ってください。

#@markdown ​

#@markdown SOURCE_SPEAKER_ID
#@markdown ソース話者のID

#@markdown SOURCE_WAVFILE
#@markdown ソース話者の音声ファイルのパス
#@markdown 学習に使用した音声ファイルのうちいずれか一つを指定してください。

#@markdown TARGET_ID
#@markdown ターゲット話者のID

#@markdown ​

#@markdown 設定が完了したら、次へを押してください。
#@markdown 変換が行われ、変換後の音声が出力されます。

#@markdown VCの性能が悪い場合、学習不足か他に問題があります。


def first_view():
  #widgetsの構成
  global SOURCE_SPEAKER_ID_input #グローバル指定
  global SOURCE_WAVFILE_input #グローバル指定
  global TARGET_ID_input #グローバル指定
  global f0_scale_input #グローバル指定
  SOURCE_SPEAKER_ID_input = widgets.IntText(value='0', placeholder='0', description='SOURCE_SPEAKER_ID:', disabled=False)
  SOURCE_WAVFILE_input = widgets.Text(value='dataset/00_myvoice/wav/VOICEACTRESS100_001.wav', placeholder='dataset/00_myvoice/wav/VOICEACTRESS100_001.wav', description='SOURCE_WAVFILE:', disabled=False)
  TARGET_ID_input = widgets.IntText(value='101', placeholder='101', description='TARGET_ID:', disabled=False)
  f0_scale_input = widgets.FloatSlider(value='2.0', min='0.1', max='5.0', step='0.1', description='f0_scale:', disabled=False)
  next_1 = widgets.Button(description='次へ', disabled=False)
  #widgetsの表示
  display(SOURCE_SPEAKER_ID_input, SOURCE_WAVFILE_input, TARGET_ID_input, f0_scale_input, next_1)
  #ボタンがクリックされたらmainを動かす
  next_1.on_click(main)

def main(b: widgets.Button) -> None:
  clear_output(True)
  #入力を変数に入れておく
  global SOURCE_SPEAKER_ID #グローバル指定
  SOURCE_SPEAKER_ID = SOURCE_SPEAKER_ID_input.value
  global SOURCE_WAVFILE #グローバル指定
  SOURCE_WAVFILE = SOURCE_WAVFILE_input.value
  global SOURCE_WAVFILE_id #グローバル指定
  SOURCE_WAVFILE_id = SOURCE_WAVFILE.split("/")[1]
  global SOURCE_WAVFILE_filename #グローバル指定
  SOURCE_WAVFILE_filename = SOURCE_WAVFILE.split("/")[-1].split(".")[0]
  global TARGET_ID #グローバル指定
  TARGET_ID = TARGET_ID_input.value
  global DUMMY #グローバル指定
  DUMMY = "dataset_etc/units/" + SOURCE_WAVFILE_id + "/" + SOURCE_WAVFILE_filename + ".npy"
  global F0 #グローバル指定
  F0 = "dataset_etc/F0/" + SOURCE_WAVFILE_id + "/" + SOURCE_WAVFILE_filename + ".npy"
  global CF0 #グローバル指定
  CF0 = "dataset_etc/cF0/" + SOURCE_WAVFILE_id + "/" + SOURCE_WAVFILE_filename + ".npy"
  global f0_scale #グローバル指定
  f0_scale = f0_scale_input.value

  hubert = torch.hub.load("bshall/hubert:main", "hubert_soft")
  with torch.no_grad():
    source, sr = torchaudio.load(SOURCE_WAVFILE)
    source = source.unsqueeze(0)
    units = hubert.units(source).numpy().squeeze(0)
    f0 = create_dataset.get_f0(SOURCE_WAVFILE, frame_length=512, win_length=256, hop_length=128)
    _, cf0, _ = create_dataset.convert_continuos_f0(f0)
    cf0_mean = np.mean(cf0)
    text = torch.FloatTensor(units)
    sid = torch.LongTensor([SOURCE_SPEAKER_ID])
    dataset = TextAudioSpeakerLoader(hps.data.training_files_notext, hps.data)
    spec, wav = dataset.get_audio(SOURCE_WAVFILE)
    f0 = torch.FloatTensor(f0)
    cf0 = torch.FloatTensor(cf0)
    data = (text, spec, wav, sid, f0, cf0)
    data = TextAudioSpeakerCollate(
      sample_rate = hps.data.sampling_rate,
      segment_size = hps.train.segment_size,
      hop_size = hps.data.hop_length,
      df_f0_type = hps.data.df_f0_type,
      dense_factors = hps.data.dense_factors,
      upsample_scales = hps.model.upsample_rates,
      sine_amp = hps.data.sine_amp,
      noise_amp = hps.data.noise_amp,
      sine_f0_type = hps.data.sine_f0_type,
      signal_types = hps.data.signal_types,
      train = False,
      f0_factor = f0_scale,
  )([data])
    x, x_lengths, spec, spec_lengths, y, y_lengths, sid_src, f0, f0_lengths, sin, d, slice_id = [x for x in data]
    sid_tgt1 = torch.LongTensor([TARGET_ID])
    audio1 = net_g.voice_conversion(spec, spec_lengths, , sid_src=sid_src, sid_tgt=sid_tgt1)[0,0].data.cpu().float().numpy()
    print("Original SID: %d" % sid_src.item())
    ipd.display(ipd.Audio(y[0].cpu().numpy(), rate=hps.data.sampling_rate,normalize = False))
    print("Converted SID: %d" % sid_tgt1.item())
    ipd.display(ipd.Audio(audio1, rate=hps.data.sampling_rate,normalize = False))

#実行
first_view()

​

## サポート専用

**以下はお問い合わせの際、指示があった場合のみ使用してください。**

In [ ]:
#@markdown **このセルは無視してください。**

#@markdown このセルは、セルが一括で実行されることを防ぐためのものです。

#@markdown  実行してしまった場合は、左側のアイコンをクリックしてセルを終了してください。

#一括実行の阻止
import time
time.sleep(86400)

​

In [ ]:
#@title ## サポート用ファイルの作成

#@markdown セルを実行すると内部処理が行われ、zipファイルが操作中のPC(またはタブレットなど)にダウンロードされます。

#@markdown ダウンロードされるzipファイルには、以下のファイルや情報が含まれます。

#@markdown * MMVC_Trainerフォルダ内のファイルの一覧
#@markdown * このノートブックで使用されている変数のリスト
#@markdown * このセッション内の出力
#@markdown * configsフォルダ内及びlogsフォルダ内のファイル
#@markdown * version.txt (MMVCのバージョンが記載されているファイル)

#@markdown これらには、ユーザーの個人情報が含まれる可能性があります。ダウンロード完了後、ファイルを共有する前に、必ず内容をご確認ください。

#@markdown ファイルのダウンロードが完了したら、ランタイムを切断してください。

#ファイルの準備
variable_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-variable.txt"
tree_dic_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-tree_dic.txt"
version_txt = "/mmvc-debug/mmvc-" + str(nowt) + "-version_txt.txt"
export_zip = "/mmvc-debug-" + str(nowt)
export_zipp = "/mmvc-debug-" + str(nowt) + ".zip"

#変数の値を保存
#whos使うと長い文字列が省略されるため、変数毎に取得する
##変数の一覧を取得
vlist = %who_ls
##それぞれの変数で値を取得してファイルに保存
with open(variable_txt, 'w') as f:
  for ev in vlist:
    try:
      #変数名(str)
      print(ev, end=' : ', file=f) 
      #変数の型(変数名がstrとなっているためevalでkeyに直す)
      print(type(eval(ev)), end=' : ', file=f)
      #変数の内容(変数名がstrとなっているためevalでkeyに直す)
      print(eval(ev), file=f)
    except:
      pass

#tree
!apt install tree
##ディレクトリ内以下
import traceback
with open(tree_dic_txt, 'w') as f:
  try:
    !tree {path} > {tree_dic_txt}
  except Exception as e:
    print("An error occurred!", file=f)
    print(e, file=f)
    print(traceback.print_exc(), file=f)

#version.txtの保存
try:
  !cp {path}/version.txt {version_txt}
except:
  pass

#logsとconfigsの保存
#directoryが無かったらmkdirしてno-dic.txt置く
!if [ -d {path}/logs ]; then if [ -z "$(ls {path}/logs)" ]; then touch {path}/logs/no-file.txt;else find . -name "*.log" -exec cp {} ~/mmvc-debug/logs \;; fi;else mkdir /mmvc-debug/logs && touch /mmvc-debug/logs/no-dic.txt; fi
!if [ -d {path}/configs ]; then if [ -z "$(ls {path}/configs)" ]; then touch {path}/configs/no-file.txt;else cp -rp {path}/configs /mmvc-debug/configs; fi;else mkdir /mmvc-debug/configs && touch /mmvc-debug/configs/no-dic.txt; fi

#直近のtracebackの保存
with open('/mmvc-debug/traceback.txt', 'w') as f:
  try:
    print(sys.last_type, sys.last_value, sys.last_traceback, file=f)
  except:
    pass

#ccaptureの保存
!mkdir /mmvc-debug/ccapture
with open('/mmvc-debug/ccapture/one_prepare_notebook.txt', 'w') as f:
  try:
    print(one_prepare_notebook, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/two_prepare_repo.txt', 'w') as f:
  try:
    print(two_prepare_repo, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/three_install_library.txt', 'w') as f:
  try:
    print(three_install_library, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/four_load_model.txt', 'w') as f:
  try:
    print(four_load_model, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/five_vc.txt', 'w') as f:
  try:
    print(five_vc, file=f)
  except:
    pass

with open('/mmvc-debug/ccapture/six_vc.txt', 'w') as f:
  try:
    print(six_vc, file=f)
  except:
    pass

#zipにまとめる
!apt install zip
!zip {export_zip} -r /mmvc-debug

#colabのfilesモジュールを使ってダウンロード
from google.colab import files
files.download(export_zipp)